<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-3/Chapter4_Empirical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install
!pip install stargazer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import
from google.colab import auth
# from google.cloud import bigquery
# from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/4 Vaccination and Election Results/Data'
os.chdir(path)

# project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
# location = 'US' # Location inserted based on the query results selected to explore
# client = bigquery.Client(project=project, location=location)
# data_table.enable_dataframe_formatter()
# auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# from zipfile import ZipFile

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

# auth.authenticate_user()
# project_id = "phd-dissertation-343113"
# !gcloud config set project {project_id}

Mounted at /content/drive


In [3]:
df = pd.read_csv('df.csv')
df.head(10)

,id_municipio,id_municipio_tse,Name,State,Region,Population,Dose1 Applied,Dose2 Applied,Dose3 Applied,Dose4 Applied,...,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,No religion,Education 8-10y,Education 11-14y,Education 15y plus,Income (BRL),Education 8y less
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,0.778335,0.655001,0.103260,0.000266,...,0.642388,0.114319,0.165616,0.000000,0.121069,10.13,6.52,0.74,1310.78,81.23
1,1100023,78,Ariquemes,RO,Norte,111148,0.660786,0.528008,0.110654,0.003194,...,0.329021,0.054720,0.179976,0.000000,0.163602,15.57,10.28,1.81,1849.30,71.52
2,1100031,450,Cabixi,RO,Norte,5067,0.881389,0.737320,0.114466,0.008881,...,0.761200,0.060193,0.288534,0.000000,0.113479,9.87,5.25,0.29,1232.03,82.53
3,1100049,94,Cacoal,RO,Norte,86416,0.845110,0.635600,0.107816,0.001365,...,0.388065,0.130994,0.184804,0.000000,0.117525,13.75,11.12,1.97,1853.64,71.00
4,1100056,272,Cerejeiras,RO,Norte,16088,0.781701,0.648993,0.145699,0.001740,...,0.627797,0.100945,0.180756,0.000000,0.090627,12.36,8.83,0.64,1573.62,76.52
5,1100064,230,Colorado do Oeste,RO,Norte,15213,0.859265,0.678104,0.112798,0.001775,...,0.704792,0.058437,0.254190,0.000000,0.098666,12.94,10.23,1.32,1518.89,73.01
6,1100072,655,Corumbiara,RO,Norte,7052,0.832388,0.724759,0.155559,0.000425,...,0.823880,0.057856,0.233976,0.000000,0.106069,5.95,2.95,0.34,1188.16,89.41
7,1100080,213,Costa Marques,RO,Norte,19255,0.461698,0.368424,0.060608,0.000675,...,0.448195,0.061023,0.123448,0.000208,0.034329,9.72,6.54,1.31,1225.56,81.35
8,1100098,256,Espigão D'Oeste,RO,Norte,33009,0.691690,0.530128,0.053379,0.000515,...,0.382805,0.208004,0.129540,0.000000,0.106304,10.91,6.00,0.73,1455.66,79.78
9,1100106,19,Guajará-Mirim,RO,Norte,46930,0.634626,0.439740,0.041168,0.000384,...,0.512849,0.093394,0.132325,0.000192,0.083273,17.19,12.16,2.32,1646.55,64.65


In [4]:
df['Outer wing 18'] = df['Far-left 18'] + df['Far-right18']
df['outer wing 22'] = df['Far-left 22'] + df['Far-right 22']

In [5]:
corr = df.corr(method='pearson')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

<ipython-input-5-6b759c8a6ef7>:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,id_municipio,id_municipio_tse,Population,Dose1 Applied,Dose2 Applied,Dose3 Applied,Dose4 Applied,Dose1 Received,Dose2 Received,Dose3 Received,Dose4 Received,Doses Applied,Doses Received,Bolsonaro Share 18,Bolsonaro Share 22,Far-left 18,Far-right18,PSL coligation 18,Far-left 22,Far-right 22,PL coligation 22,Vaccination coverage,BCG coverage,Polio coverage,elderly(>70yo),White,Brown,Pop. density,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,No religion,Education 8-10y,Education 11-14y,Income (BRL),Education 8y less,Outer wing 18,outer wing 22
id_municipio,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
id_municipio_tse,0.86,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Population,-0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose1 Applied,0.39,0.38,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose2 Applied,0.44,0.43,-0.01,0.92,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Applied,0.39,0.40,-0.02,0.56,0.73,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose4 Applied,0.03,0.05,0.00,0.07,0.15,0.40,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose1 Received,0.36,0.32,-0.03,0.65,0.70,0.55,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose2 Received,0.42,0.39,-0.03,0.66,0.81,0.71,0.17,0.94,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Received,0.40,0.42,-0.02,0.49,0.68,0.96,0.39,0.64,0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Far-left 22', 'Far-right 22',
       'PL coligation 22', 'Vaccination coverage', 'BCG coverage',
       'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black',
       'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)', 'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [7]:
y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage'
x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
# x_vars1 = ('PSL coligation 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

x_vars4 = ('PSL coligation 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

x_vars5 = ('PL coligation 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

In [8]:
# y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received'
# x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'White', 'Brown', 'Pop. density', 'Evangelic',
#            'No religion', 'Education 8y less', 'Education 11-14y')
# x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'White', 'Brown', 'Pop. density', 'Evangelic',
#            'No religion', 'Education 8y less', 'Education 11-14y')
# x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'White', 'Brown', 'Pop. density', 'Evangelic',
#            'No religion', 'Education 8y less', 'Education 11-14y')

In [9]:
df[list(x_vars1)] = df[list(x_vars1)].replace('-', 0).astype('float64')

In [10]:
models = []

In [11]:
# setting DF for OLS
X1 = sm.add_constant(df[list(x_vars1)])
X2 = sm.add_constant(df[list(x_vars2)])
X3 = sm.add_constant(df[list(x_vars3)])
X4 = sm.add_constant(df[list(x_vars4)])
X5 = sm.add_constant(df[list(x_vars5)])

# target1 (y1) as Doses Applied
y1 = df[y_vars[0]].values.reshape(-1, 1)

# target2 (y2) as Doses Received
y2 = df[y_vars[1]].values.reshape(-1, 1)

# target3 (y3) as Dose3 Applied
y3 = df[y_vars[2]].values.reshape(-1, 1)

# target4 (y24 as Dose3 Received
y4 = df[y_vars[3]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5 = df[y_vars[4]].values.reshape(-1, 1)

# target6 (y6) as Polio vaxx coverage
y6 = df[y_vars[5]].values.reshape(-1, 1)



In [12]:
# Doses aplicadas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X1).fit()

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     283.7
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:16   Log-Likelihood:                -1227.7
No. Observations:                5507   AIC:                             2491.
Df Residuals:                    5489   BIC:                             2610.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.6342      0.561     -2.915      0.004      -2.733      -0.535
Bolsonaro Share 22       -0.0983      0.046     -2.147      0.032      -0.188      -0.009
elderly(>70yo)            0.0471      0.003     14.608      0.000       0.041       0.053
Rural pop.             1.076e-05      0.000      0.037      0.970      -0.001       0.001
White                    -0.0227      0.005     -4.948      0.000      -0.032      -0.014
Black                    -0.0244      0.005     -5.037      0.000      -0.034      -0.015
Brown                    -0.0280      0.005     -6.035      0.000      -0.037      -0.019
Indigenous               -0.0263      0.005     -5.642      0.000      -0.035      -0.017
Pop. density          -7.828e-06   7.73e-06     -1.013      0.311    -2.3e-05    7.32e-06
Catholic                  1.0153      0.036     27.866      0.000       0.944       1.087
Evangelic                 0.5988      0.092      6.501      0.000       0.418       0.779
Pentecostal Evangelic     1.0157      0.085     11.977      0.000       0.849       1.182
No religion               1.1705      0.116     10.067      0.000       0.943       1.398
Education 8y less         0.0424      0.004     11.440      0.000       0.035       0.050
Education 8-10y           0.0485      0.004     11.747      0.000       0.040       0.057
Education 11-14y          0.0524      0.004     12.692      0.000       0.044       0.060
Education 15y plus        0.0147      0.006      2.340      0.019       0.002       0.027
Income (BRL)              0.0002   1.85e-05      8.575      0.000       0.000       0.000
==============================================================================
Omnibus:                     3539.639   Durbin-Watson:                   1.450
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           549979.552
Skew:                           2.085   Prob(JB):                         0.00
Kurtosis:                      51.780   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# Doses recebidas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X1).fit()

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     430.0
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:16   Log-Likelihood:                -319.23
No. Observations:                5507   AIC:                             674.5
Df Residuals:                    5489   BIC:                             793.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3094      0.475      4.858      0.000       1.377       3.241
Bolsonaro Share 22       -0.1237      0.039     -3.188      0.001      -0.200      -0.048
elderly(>70yo)            0.0647      0.003     23.683      0.000       0.059       0.070
Rural pop.               -0.0012      0.000     -4.994      0.000      -0.002      -0.001
White                    -0.0204      0.004     -5.264      0.000      -0.028      -0.013
Black                    -0.0259      0.004     -6.295      0.000      -0.034      -0.018
Brown                    -0.0249      0.004     -6.338      0.000      -0.033      -0.017
Indigenous               -0.0236      0.004     -5.974      0.000      -0.031      -0.016
Pop. density          -6.998e-06   6.55e-06     -1.068      0.286   -1.98e-05    5.85e-06
Catholic                  1.1891      0.031     38.487      0.000       1.129       1.250
Evangelic                 0.6574      0.078      8.418      0.000       0.504       0.810
Pentecostal Evangelic     1.0957      0.072     15.237      0.000       0.955       1.237
No religion               1.1229      0.099     11.390      0.000       0.930       1.316
Education 8y less        -0.0009      0.003     -0.291      0.771      -0.007       0.005
Education 8-10y           0.0096      0.004      2.751      0.006       0.003       0.016
Education 11-14y          0.0071      0.003      2.019      0.044       0.000       0.014
Education 15y plus       -0.0289      0.005     -5.429      0.000      -0.039      -0.018
Income (BRL)              0.0001   1.57e-05      8.529      0.000       0.000       0.000
==============================================================================
Omnibus:                      764.597   Durbin-Watson:                   1.825
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3799.809
Skew:                           0.577   Prob(JB):                         0.00
Kurtosis:                       6.902   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
# 3ª dose aplicada ~ Bolsonaro Share '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X1).fit()

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     264.5
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:16   Log-Likelihood:                 7531.6
No. Observations:                5507   AIC:                        -1.503e+04
Df Residuals:                    5489   BIC:                        -1.491e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3876      0.114     -3.392      0.001      -0.612      -0.164
Bolsonaro Share 22       -0.0284      0.009     -3.044      0.002      -0.047      -0.010
elderly(>70yo)            0.0176      0.001     26.717      0.000       0.016       0.019
Rural pop.                0.0002   5.86e-05      3.973      0.000       0.000       0.000
White                    -0.0009      0.001     -1.001      0.317      -0.003       0.001
Black                    -0.0023      0.001     -2.361      0.018      -0.004      -0.000
Brown                    -0.0019      0.001     -2.001      0.045      -0.004   -3.85e-05
Indigenous               -0.0010      0.001     -1.097      0.273      -0.003       0.001
Pop. density          -1.802e-06   1.57e-06     -1.144      0.253   -4.89e-06    1.29e-06
Catholic                  0.0861      0.007     11.598      0.000       0.072       0.101
Evangelic                 0.0068      0.019      0.363      0.717      -0.030       0.044
Pentecostal Evangelic     0.1221      0.017      7.064      0.000       0.088       0.156
No religion               0.1121      0.024      4.730      0.000       0.066       0.159
Education 8y less         0.0044      0.001      5.853      0.000       0.003       0.006
Education 8-10y           0.0058      0.001      6.946      0.000       0.004       0.007
Education 11-14y          0.0062      0.001      7.317      0.000       0.005       0.008
Education 15y plus       -0.0007      0.001     -0.543      0.587      -0.003       0.002
Income (BRL)           2.714e-05   3.78e-06      7.188      0.000    1.97e-05    3.45e-05
==============================================================================
Omnibus:                     2078.696   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23353.099
Skew:                           1.480   Prob(JB):                         0.00
Kurtosis:                      12.644   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
# 3ª dose recebida ~ Bolsonaro Share '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X1).fit()

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     327.2
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:16   Log-Likelihood:                 7999.2
No. Observations:                5507   AIC:                        -1.596e+04
Df Residuals:                    5489   BIC:                        -1.584e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2364      0.105     -2.253      0.024      -0.442      -0.031
Bolsonaro Share 22       -0.0200      0.009     -2.335      0.020      -0.037      -0.003
elderly(>70yo)            0.0177      0.001     29.376      0.000       0.017       0.019
Rural pop.                0.0001   5.38e-05      2.379      0.017    2.25e-05       0.000
White                    -0.0010      0.001     -1.128      0.259      -0.003       0.001
Black                    -0.0024      0.001     -2.668      0.008      -0.004      -0.001
Brown                    -0.0019      0.001     -2.210      0.027      -0.004      -0.000
Indigenous               -0.0010      0.001     -1.153      0.249      -0.003       0.001
Pop. density          -1.178e-06   1.45e-06     -0.814      0.416   -4.01e-06    1.66e-06
Catholic                  0.0996      0.007     14.596      0.000       0.086       0.113
Evangelic                 0.0145      0.017      0.844      0.399      -0.019       0.048
Pentecostal Evangelic     0.1229      0.016      7.742      0.000       0.092       0.154
No religion               0.1130      0.022      5.189      0.000       0.070       0.156
Education 8y less         0.0028      0.001      4.094      0.000       0.001       0.004
Education 8-10y           0.0043      0.001      5.595      0.000       0.003       0.006
Education 11-14y          0.0047      0.001      6.043      0.000       0.003       0.006
Education 15y plus       -0.0022      0.001     -1.907      0.057      -0.005    6.32e-05
Income (BRL)           2.496e-05   3.47e-06      7.197      0.000    1.82e-05    3.18e-05
==============================================================================
Omnibus:                     1512.637   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9820.993
Skew:                           1.151   Prob(JB):                         0.00
Kurtosis:                       9.124   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# Cobertura vacinal ~ Bolsonaro Share '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X1).fit()

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.191
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     76.39
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          1.38e-237
Time:                        18:51:16   Log-Likelihood:                -24774.
No. Observations:                5507   AIC:                         4.958e+04
Df Residuals:                    5489   BIC:                         4.970e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   151.8909     40.323      3.767      0.000      72.841     230.940
Bolsonaro Share 22       16.2145      3.292      4.925      0.000       9.761      22.668
elderly(>70yo)            2.4174      0.232     10.428      0.000       1.963       2.872
Rural pop.               -0.0162      0.021     -0.782      0.434      -0.057       0.024
White                    -1.1944      0.329     -3.625      0.000      -1.840      -0.548
Black                    -1.3412      0.349     -3.846      0.000      -2.025      -0.658
Brown                    -1.3258      0.333     -3.977      0.000      -1.979      -0.672
Indigenous               -1.4714      0.336     -4.385      0.000      -2.129      -0.814
Pop. density             -0.0014      0.001     -2.497      0.013      -0.002      -0.000
Catholic                 11.2929      2.621      4.309      0.000       6.155      16.431
Evangelic                -4.5235      6.624     -0.683      0.495     -17.510       8.463
Pentecostal Evangelic     7.0298      6.100      1.152      0.249      -4.928      18.987
No religion             -55.5050      8.363     -6.637      0.000     -71.900     -39.110
Education 8y less         0.2022      0.267      0.758      0.449      -0.321       0.725
Education 8-10y           0.2240      0.297      0.754      0.451      -0.358       0.806
Education 11-14y         -0.3664      0.297     -1.235      0.217      -0.948       0.215
Education 15y plus       -2.7009      0.451     -5.991      0.000      -3.585      -1.817
Income (BRL)              0.0100      0.001      7.483      0.000       0.007       0.013
==============================================================================
Omnibus:                      545.195   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2130.295
Skew:                           0.439   Prob(JB):                         0.00
Kurtosis:                       5.917   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
# Cobertura contra Polio ~ Bolsonaro share '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X1).fit()

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     46.17
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          3.71e-145
Time:                        18:51:16   Log-Likelihood:                -26168.
No. Observations:                5507   AIC:                         5.237e+04
Df Residuals:                    5489   BIC:                         5.249e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   190.2416     51.946      3.662      0.000      88.407     292.076
Bolsonaro Share 22       12.4666      4.241      2.940      0.003       4.153      20.780
elderly(>70yo)            2.8601      0.299      9.577      0.000       2.275       3.446
Rural pop.               -0.0145      0.027     -0.545      0.586      -0.067       0.038
White                    -1.2818      0.424     -3.020      0.003      -2.114      -0.450
Black                    -1.4982      0.449     -3.335      0.001      -2.379      -0.618
Brown                    -1.4417      0.429     -3.357      0.001      -2.284      -0.600
Indigenous               -1.7050      0.432     -3.944      0.000      -2.552      -0.857
Pop. density             -0.0010      0.001     -1.440      0.150      -0.002       0.000
Catholic                 11.6948      3.376      3.464      0.001       5.076      18.313
Evangelic               -13.1078      8.534     -1.536      0.125     -29.837       3.621
Pentecostal Evangelic     5.1778      7.858      0.659      0.510     -10.227      20.582
No religion             -44.0390     10.774     -4.088      0.000     -65.159     -22.919
Education 8y less         0.0649      0.344      0.189      0.850      -0.609       0.739
Education 8-10y           0.0127      0.383      0.033      0.974      -0.737       0.763
Education 11-14y         -0.5517      0.382     -1.443      0.149      -1.301       0.198
Education 15y plus       -2.8115      0.581     -4.841      0.000      -3.950      -1.673
Income (BRL)              0.0081      0.002      4.718      0.000       0.005       0.011
==============================================================================
Omnibus:                     2158.208   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35014.909
Skew:                           1.438   Prob(JB):                         0.00
Kurtosis:                      15.014   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
# Doses aplicadas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model1b = sm.OLS(y1, X2).fit()

# Add the model name to the list of models
models.append(model1b)

model1b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     267.8
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:16   Log-Likelihood:                -1228.0
No. Observations:                5507   AIC:                             2494.
Df Residuals:                    5488   BIC:                             2620.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.6070      0.566     -2.840      0.005      -2.716      -0.498
Far-left 18               3.7817      1.984      1.907      0.057      -0.107       7.670
Far-right 22             -1.2700      1.470     -0.864      0.388      -4.151       1.611
elderly(>70yo)            0.0486      0.003     14.978      0.000       0.042       0.055
Rural pop.            -3.707e-05      0.000     -0.129      0.898      -0.001       0.001
White                    -0.0225      0.005     -4.888      0.000      -0.031      -0.013
Black                    -0.0238      0.005     -4.893      0.000      -0.033      -0.014
Brown                    -0.0275      0.005     -5.936      0.000      -0.037      -0.018
Indigenous               -0.0261      0.005     -5.580      0.000      -0.035      -0.017
Pop. density          -7.855e-06    7.8e-06     -1.007      0.314   -2.32e-05    7.44e-06
Catholic                  1.0280      0.036     28.516      0.000       0.957       1.099
Evangelic                 0.5824      0.092      6.352      0.000       0.403       0.762
Pentecostal Evangelic     0.9667      0.082     11.860      0.000       0.807       1.126
No religion               1.1365      0.118      9.646      0.000       0.906       1.367
Education 8y less         0.0415      0.004     11.222      0.000       0.034       0.049
Education 8-10y           0.0467      0.004     11.367      0.000       0.039       0.055
Education 11-14y          0.0511      0.004     12.479      0.000       0.043       0.059
Education 15y plus        0.0151      0.006      2.393      0.017       0.003       0.028
Income (BRL)              0.0001    1.7e-05      8.478      0.000       0.000       0.000
==============================================================================
Omnibus:                     3561.744   Durbin-Watson:                   1.449
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           551480.947
Skew:                           2.108   Prob(JB):                         0.00
Kurtosis:                      51.843   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2b = sm.OLS(y2, X2).fit()

# Add the model name to the list of models
models.append(model2b)

model2b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     405.4
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:17   Log-Likelihood:                -321.51
No. Observations:                5507   AIC:                             681.0
Df Residuals:                    5488   BIC:                             806.7
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.2379      0.480      4.663      0.000       1.297       3.179
Far-left 18               1.6976      1.683      1.009      0.313      -1.601       4.996
Far-right 22             -2.8104      1.247     -2.254      0.024      -5.254      -0.366
elderly(>70yo)            0.0660      0.003     23.975      0.000       0.061       0.071
Rural pop.               -0.0013      0.000     -5.212      0.000      -0.002      -0.001
White                    -0.0195      0.004     -5.008      0.000      -0.027      -0.012
Black                    -0.0242      0.004     -5.856      0.000      -0.032      -0.016
Brown                    -0.0237      0.004     -6.030      0.000      -0.031      -0.016
Indigenous               -0.0226      0.004     -5.699      0.000      -0.030      -0.015
Pop. density          -4.055e-06   6.62e-06     -0.613      0.540    -1.7e-05    8.92e-06
Catholic                  1.2049      0.031     39.402      0.000       1.145       1.265
Evangelic                 0.6339      0.078      8.152      0.000       0.481       0.786
Pentecostal Evangelic     1.0204      0.069     14.759      0.000       0.885       1.156
No religion               1.0808      0.100     10.814      0.000       0.885       1.277
Education 8y less        -0.0016      0.003     -0.496      0.620      -0.008       0.005
Education 8-10y           0.0084      0.003      2.415      0.016       0.002       0.015
Education 11-14y          0.0060      0.003      1.715      0.086      -0.001       0.013
Education 15y plus       -0.0267      0.005     -4.988      0.000      -0.037      -0.016
Income (BRL)              0.0001   1.45e-05      8.167      0.000    8.97e-05       0.000
==============================================================================
Omnibus:                      742.387   Durbin-Watson:                   1.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3508.189
Skew:                           0.572   Prob(JB):                         0.00
Kurtosis:                       6.739   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
# 3ª dose aplicada ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model3b = sm.OLS(y3, X2).fit()

# Add the model name to the list of models
models.append(model3b)

model3b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.455
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     254.7
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:17   Log-Likelihood:                 7556.0
No. Observations:                5507   AIC:                        -1.507e+04
Df Residuals:                    5488   BIC:                        -1.495e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.4947      0.115     -4.309      0.000      -0.720      -0.270
Far-left 18              -0.1672      0.402     -0.415      0.678      -0.956       0.622
Far-right 22             -2.2398      0.298     -7.511      0.000      -2.824      -1.655
elderly(>70yo)            0.0178      0.001     27.004      0.000       0.016       0.019
Rural pop.                0.0002   5.85e-05      3.494      0.000    8.97e-05       0.000
White                    -0.0003      0.001     -0.286      0.775      -0.002       0.002
Black                    -0.0013      0.001     -1.329      0.184      -0.003       0.001
Brown                    -0.0012      0.001     -1.289      0.197      -0.003       0.001
Indigenous               -0.0004      0.001     -0.409      0.683      -0.002       0.001
Pop. density          -1.509e-07   1.58e-06     -0.095      0.924   -3.25e-06    2.95e-06
Catholic                  0.0902      0.007     12.328      0.000       0.076       0.105
Evangelic                 0.0007      0.019      0.037      0.971      -0.036       0.037
Pentecostal Evangelic     0.0961      0.017      5.810      0.000       0.064       0.128
No religion               0.0878      0.024      3.674      0.000       0.041       0.135
Education 8y less         0.0048      0.001      6.402      0.000       0.003       0.006
Education 8-10y           0.0063      0.001      7.525      0.000       0.005       0.008
Education 11-14y          0.0063      0.001      7.525      0.000       0.005       0.008
Education 15y plus        0.0008      0.001      0.615      0.538      -0.002       0.003
Income (BRL)            2.63e-05   3.46e-06      7.607      0.000    1.95e-05    3.31e-05
==============================================================================
Omnibus:                     2106.549   Durbin-Watson:                   1.793
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23788.514
Skew:                           1.503   Prob(JB):                         0.00
Kurtosis:                      12.728   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
# 3ª dose recebida ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model4b = sm.OLS(y4, X2).fit()

# Add the model name to the list of models
models.append(model4b)

model4b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     315.6
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:17   Log-Likelihood:                 8028.7
No. Observations:                5507   AIC:                        -1.602e+04
Df Residuals:                    5488   BIC:                        -1.589e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3427      0.105     -3.253      0.001      -0.549      -0.136
Far-left 18              -0.1348      0.369     -0.365      0.715      -0.859       0.589
Far-right 22             -2.1668      0.274     -7.917      0.000      -2.703      -1.630
elderly(>70yo)            0.0179      0.001     29.638      0.000       0.017       0.019
Rural pop.                0.0001   5.37e-05      1.887      0.059   -3.92e-06       0.000
White                    -0.0004      0.001     -0.410      0.682      -0.002       0.001
Black                    -0.0015      0.001     -1.658      0.097      -0.003       0.000
Brown                    -0.0013      0.001     -1.520      0.128      -0.003       0.000
Indigenous               -0.0004      0.001     -0.484      0.629      -0.002       0.001
Pop. density           1.859e-07   1.45e-06      0.128      0.898   -2.66e-06    3.03e-06
Catholic                  0.1026      0.007     15.285      0.000       0.089       0.116
Evangelic                 0.0101      0.017      0.594      0.553      -0.023       0.044
Pentecostal Evangelic     0.1019      0.015      6.711      0.000       0.072       0.132
No religion               0.0896      0.022      4.085      0.000       0.047       0.133
Education 8y less         0.0033      0.001      4.788      0.000       0.002       0.005
Education 8-10y           0.0049      0.001      6.345      0.000       0.003       0.006
Education 11-14y          0.0049      0.001      6.369      0.000       0.003       0.006
Education 15y plus       -0.0009      0.001     -0.749      0.454      -0.003       0.001
Income (BRL)           2.541e-05   3.17e-06      8.007      0.000    1.92e-05    3.16e-05
==============================================================================
Omnibus:                     1522.792   Durbin-Watson:                   1.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9800.775
Skew:                           1.162   Prob(JB):                         0.00
Kurtosis:                       9.108   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
# Cobertura vacinal geral ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model5b = sm.OLS(y5, X2).fit()

# Add the model name to the list of models
models.append(model5b)

model5b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     74.52
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          4.66e-244
Time:                        18:51:17   Log-Likelihood:                -24756.
No. Observations:                5507   AIC:                         4.955e+04
Df Residuals:                    5488   BIC:                         4.968e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   169.1307     40.563      4.170      0.000      89.611     248.651
Far-left 18            -744.2841    142.214     -5.234      0.000   -1023.080    -465.489
Far-right 22            659.3257    105.373      6.257      0.000     452.754     865.898
elderly(>70yo)            2.1229      0.233      9.126      0.000       1.667       2.579
Rural pop.               -0.0024      0.021     -0.117      0.906      -0.043       0.038
White                    -1.3221      0.330     -4.012      0.000      -1.968      -0.676
Black                    -1.5721      0.349     -4.507      0.000      -2.256      -0.888
Brown                    -1.4762      0.333     -4.436      0.000      -2.129      -0.824
Indigenous               -1.5821      0.335     -4.718      0.000      -2.239      -0.925
Pop. density             -0.0014      0.001     -2.544      0.011      -0.003      -0.000
Catholic                  9.0280      2.585      3.493      0.000       3.961      14.095
Evangelic                -1.8591      6.573     -0.283      0.777     -14.745      11.026
Pentecostal Evangelic    16.8546      5.844      2.884      0.004       5.399      28.311
No religion             -44.5360      8.447     -5.272      0.000     -61.096     -27.976
Education 8y less         0.2174      0.265      0.819      0.413      -0.303       0.737
Education 8-10y           0.3644      0.295      1.236      0.216      -0.213       0.942
Education 11-14y         -0.2457      0.294     -0.836      0.403      -0.822       0.330
Education 15y plus       -2.9741      0.453     -6.564      0.000      -3.862      -2.086
Income (BRL)              0.0116      0.001      9.457      0.000       0.009       0.014
==============================================================================
Omnibus:                      525.470   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2037.095
Skew:                           0.421   Prob(JB):                         0.00
Kurtosis:                       5.858   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
# Cobertura contra Polio ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model6b = sm.OLS(y6, X2).fit()

# Add the model name to the list of models
models.append(model6b)

model6b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     46.03
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          2.00e-152
Time:                        18:51:17   Log-Likelihood:                -26149.
No. Observations:                5507   AIC:                         5.234e+04
Df Residuals:                    5488   BIC:                         5.246e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   214.8381     52.239      4.113      0.000     112.430     317.246
Far-left 18            -800.4412    183.147     -4.370      0.000   -1159.481    -441.401
Far-right 22            786.0309    135.702      5.792      0.000     520.002    1052.060
elderly(>70yo)            2.5707      0.300      8.582      0.000       1.983       3.158
Rural pop.                0.0006      0.027      0.021      0.983      -0.052       0.053
White                    -1.4203      0.424     -3.346      0.001      -2.252      -0.588
Black                    -1.7297      0.449     -3.850      0.000      -2.610      -0.849
Brown                    -1.5882      0.429     -3.706      0.000      -2.428      -0.748
Indigenous               -1.8109      0.432     -4.194      0.000      -2.657      -0.964
Pop. density             -0.0010      0.001     -1.323      0.186      -0.002       0.000
Catholic                  9.8168      3.329      2.949      0.003       3.292      16.342
Evangelic               -11.2182      8.465     -1.325      0.185     -27.812       5.376
Pentecostal Evangelic    13.4050      7.526      1.781      0.075      -1.348      28.158
No religion             -31.6006     10.879     -2.905      0.004     -52.927     -10.274
Education 8y less        -0.0016      0.342     -0.005      0.996      -0.671       0.668
Education 8-10y           0.0546      0.380      0.144      0.886      -0.690       0.799
Education 11-14y         -0.4997      0.378     -1.321      0.187      -1.241       0.242
Education 15y plus       -3.0989      0.583     -5.311      0.000      -4.243      -1.955
Income (BRL)              0.0088      0.002      5.610      0.000       0.006       0.012
==============================================================================
Omnibus:                     2135.101   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34095.200
Skew:                           1.422   Prob(JB):                         0.00
Kurtosis:                      14.853   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
models

In [25]:
table1 = Stargazer([model1, model2, model3, model4, model5, model6])
table1.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
table2 = Stargazer([model1b, model2b, model3b, model4b, model5b, model6b ])
table2.covariate_order(['Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])

In [26]:
print(table1.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & -0.098$^{**}$ & -0.124$^{***}$ & -0.028$^{***}$ & -0.020$^{**}$ & 16.215$^{***}$ & 12.467$^{***}$ \\
  & (0.046) & (0.039) & (0.009) & (0.009) & (3.292) & (4.241) \\
 elderly(>70yo) & 0.047$^{***}$ & 0.065$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.417$^{***}$ & 2.860$^{***}$ \\
  & (0.003) & (0.003) & (0.001) & (0.001) & (0.232) & (0.299) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & -0.016$^{}$ & -0.015$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.027) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.001$^{**}$ & -0.001$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.023$^{***}$ & -0.020$^{***}$ & -0.

In [27]:
print(table2.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Far-left 18 & 3.782$^{*}$ & 1.698$^{}$ & -0.167$^{}$ & -0.135$^{}$ & -744.284$^{***}$ & -800.441$^{***}$ \\
  & (1.984) & (1.683) & (0.402) & (0.369) & (142.214) & (183.147) \\
 Far-right 22 & -1.270$^{}$ & -2.810$^{**}$ & -2.240$^{***}$ & -2.167$^{***}$ & 659.326$^{***}$ & 786.031$^{***}$ \\
  & (1.470) & (1.247) & (0.298) & (0.274) & (105.373) & (135.702) \\
 elderly(>70yo) & 0.049$^{***}$ & 0.066$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.123$^{***}$ & 2.571$^{***}$ \\
  & (0.003) & (0.003) & (0.001) & (0.001) & (0.233) & (0.300) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{*}$ & -0.002$^{}$ & 0.001$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.027) \\
 Pop. density & -0.000$^{}$ & -0.000$^{

In [28]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [29]:
# setting DF for OLS
df_co = df[df['Region'] == 'Centro-Oeste']
df_ne = df[df['Region'] == 'Nordeste']
df_no = df[df['Region'] == 'Norte']
df_se = df[df['Region'] == 'Sudeste']
df_su = df[df['Region'] == 'Sul']

X2_co = sm.add_constant(df_co[list(x_vars1)])
X2_ne = sm.add_constant(df_ne[list(x_vars1)])
X2_no = sm.add_constant(df_no[list(x_vars1)])
X2_se = sm.add_constant(df_se[list(x_vars1)])
X2_su = sm.add_constant(df_su[list(x_vars1)])

# target2 (y2) as Doses Received
y2_co = df_co[y_vars[1]].values.reshape(-1, 1)
y2_ne = df_ne[y_vars[1]].values.reshape(-1, 1)
y2_no = df_no[y_vars[1]].values.reshape(-1, 1)
y2_se = df_se[y_vars[1]].values.reshape(-1, 1)
y2_su = df_su[y_vars[1]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5_co = df_co[y_vars[4]].values.reshape(-1, 1)
y5_ne = df_ne[y_vars[4]].values.reshape(-1, 1)
y5_no = df_no[y_vars[4]].values.reshape(-1, 1)
y5_se = df_se[y_vars[4]].values.reshape(-1, 1)
y5_su = df_su[y_vars[4]].values.reshape(-1, 1)

In [30]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y2_co, X2_co).fit()

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     39.94
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           3.08e-77
Time:                        18:51:17   Log-Likelihood:                -14.181
No. Observations:                 446   AIC:                             64.36
Df Residuals:                     428   BIC:                             138.2
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.8949      2.985      1.975      0.049       0.027      11.763
Bolsonaro Share 22       -0.6382      0.172     -3.705      0.000      -0.977      -0.300
elderly(>70yo)            0.0152      0.012      1.268      0.206      -0.008       0.039
Rural pop.            -7.624e-05      0.001     -0.075      0.940      -0.002       0.002
White                    -0.0265      0.014     -1.857      0.064      -0.055       0.002
Black                    -0.0529      0.016     -3.355      0.001      -0.084      -0.022
Brown                    -0.0341      0.014     -2.393      0.017      -0.062      -0.006
Indigenous               -0.0327      0.014     -2.302      0.022      -0.061      -0.005
Pop. density             -0.0002   8.13e-05     -2.249      0.025      -0.000   -2.31e-05
Catholic                  1.4086      0.093     15.137      0.000       1.226       1.592
Evangelic                -0.4809      0.597     -0.805      0.421      -1.655       0.693
Pentecostal Evangelic     1.8248      0.204      8.938      0.000       1.423       2.226
No religion               1.0100      0.363      2.785      0.006       0.297       1.723
Education 8y less        -0.0264      0.027     -0.994      0.321      -0.079       0.026
Education 8-10y          -0.0184      0.027     -0.676      0.499      -0.072       0.035
Education 11-14y         -0.0096      0.027     -0.358      0.720      -0.062       0.043
Education 15y plus       -0.0563      0.034     -1.639      0.102      -0.124       0.011
Income (BRL)              0.0002   6.48e-05      3.781      0.000       0.000       0.000
==============================================================================
Omnibus:                       37.997   Durbin-Watson:                   2.095
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.789
Skew:                           0.581   Prob(JB):                     1.04e-13
Kurtosis:                       4.367   Cond. No.                     4.00e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
# Create the OLS model with a different name
model2ne = sm.OLS(y2_ne, X2_ne).fit()

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     100.1
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          6.15e-244
Time:                        18:51:17   Log-Likelihood:                -35.961
No. Observations:                1787   AIC:                             107.9
Df Residuals:                    1769   BIC:                             206.7
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3964      0.662      3.618      0.000       1.097       3.695
Bolsonaro Share 22       -0.3977      0.094     -4.215      0.000      -0.583      -0.213
elderly(>70yo)            0.0592      0.005     11.245      0.000       0.049       0.069
Rural pop.               -0.0011      0.000     -2.587      0.010      -0.002      -0.000
White                    -0.0135      0.006     -2.315      0.021      -0.025      -0.002
Black                    -0.0163      0.006     -2.613      0.009      -0.029      -0.004
Brown                    -0.0198      0.006     -3.367      0.001      -0.031      -0.008
Indigenous               -0.0150      0.006     -2.459      0.014      -0.027      -0.003
Pop. density          -2.603e-05   1.77e-05     -1.473      0.141   -6.07e-05    8.62e-06
Catholic                  1.5246      0.058     26.467      0.000       1.412       1.638
Evangelic                 0.1004      0.357      0.281      0.778      -0.600       0.800
Pentecostal Evangelic     1.0281      0.175      5.877      0.000       0.685       1.371
No religion               1.6493      0.160     10.303      0.000       1.335       1.963
Education 8y less        -0.0109      0.004     -2.754      0.006      -0.019      -0.003
Education 8-10y           0.0046      0.005      0.866      0.386      -0.006       0.015
Education 11-14y         -0.0128      0.005     -2.419      0.016      -0.023      -0.002
Education 15y plus       -0.0052      0.013     -0.392      0.695      -0.031       0.021
Income (BRL)              0.0003   4.93e-05      5.344      0.000       0.000       0.000
==============================================================================
Omnibus:                      161.024   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              848.179
Skew:                           0.234   Prob(JB):                    6.61e-185
Kurtosis:                       6.342   Cond. No.                     1.04e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
# Create the OLS model with a different name
model2no = sm.OLS(y2_no, X2_no).fit()

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     34.20
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           4.95e-69
Time:                        18:51:17   Log-Likelihood:                 26.497
No. Observations:                 449   AIC:                            -16.99
Df Residuals:                     431   BIC:                             56.93
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.3699      1.843      0.201      0.841      -3.253       3.992
Bolsonaro Share 22       -0.1404      0.112     -1.256      0.210      -0.360       0.079
elderly(>70yo)            0.0912      0.013      6.881      0.000       0.065       0.117
Rural pop.                0.0015      0.001      1.764      0.078      -0.000       0.003
White                     0.0039      0.013      0.303      0.762      -0.022       0.030
Black                    -0.0116      0.014     -0.828      0.408      -0.039       0.016
Brown                    -0.0070      0.013     -0.536      0.592      -0.032       0.019
Indigenous               -0.0050      0.013     -0.383      0.702      -0.031       0.021
Pop. density          -8.292e-05   8.43e-05     -0.984      0.326      -0.000    8.27e-05
Catholic                  0.9070      0.090     10.064      0.000       0.730       1.084
Evangelic                 0.1322      0.407      0.324      0.746      -0.669       0.933
Pentecostal Evangelic     0.2794      0.185      1.507      0.133      -0.085       0.644
No religion               0.6448      0.304      2.122      0.034       0.048       1.242
Education 8y less        -0.0010      0.013     -0.078      0.938      -0.027       0.025
Education 8-10y           0.0174      0.015      1.183      0.237      -0.012       0.046
Education 11-14y         -0.0029      0.015     -0.200      0.842      -0.032       0.026
Education 15y plus        0.0384      0.032      1.212      0.226      -0.024       0.101
Income (BRL)              0.0002   7.56e-05      2.802      0.005    6.32e-05       0.000
==============================================================================
Omnibus:                       81.923   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              267.107
Skew:                           0.818   Prob(JB):                     9.97e-59
Kurtosis:                       6.406   Cond. No.                     1.97e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
# Create the OLS model with a different name
model2se = sm.OLS(y2_se, X2_se).fit()

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     60.54
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          1.02e-159
Time:                        18:51:17   Log-Likelihood:                -79.096
No. Observations:                1666   AIC:                             194.2
Df Residuals:                    1648   BIC:                             291.7
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.8317      1.331      2.128      0.034       0.221       5.442
Bolsonaro Share 22       -0.0535      0.092     -0.580      0.562      -0.234       0.127
elderly(>70yo)            0.0561      0.005     10.264      0.000       0.045       0.067
Rural pop.               -0.0003      0.001     -0.610      0.542      -0.001       0.001
White                    -0.0148      0.008     -1.920      0.055      -0.030       0.000
Black                    -0.0250      0.008     -3.126      0.002      -0.041      -0.009
Brown                    -0.0202      0.008     -2.580      0.010      -0.036      -0.005
Indigenous               -0.0182      0.008     -2.144      0.032      -0.035      -0.002
Pop. density           -3.17e-06   7.66e-06     -0.414      0.679   -1.82e-05    1.19e-05
Catholic                  1.1144      0.094     11.896      0.000       0.931       1.298
Evangelic                 0.8186      0.182      4.488      0.000       0.461       1.176
Pentecostal Evangelic     1.0299      0.163      6.325      0.000       0.711       1.349
No religion               0.9620      0.261      3.685      0.000       0.450       1.474
Education 8y less        -0.0101      0.010     -0.969      0.333      -0.031       0.010
Education 8-10y           0.0009      0.011      0.083      0.934      -0.021       0.023
Education 11-14y         -0.0033      0.011     -0.308      0.758      -0.024       0.018
Education 15y plus       -0.0619      0.012     -5.008      0.000      -0.086      -0.038
Income (BRL)              0.0002   2.97e-05      5.815      0.000       0.000       0.000
==============================================================================
Omnibus:                      224.867   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              550.992
Skew:                           0.754   Prob(JB):                    2.26e-120
Kurtosis:                       5.380   Cond. No.                     3.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [34]:
# Create the OLS model with a different name
model2su = sm.OLS(y2_su, X2_su).fit()

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     82.17
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          3.48e-184
Time:                        18:51:17   Log-Likelihood:                 159.51
No. Observations:                1159   AIC:                            -283.0
Df Residuals:                    1141   BIC:                            -192.0
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.7986      1.245     -0.642      0.521      -3.241       1.643
Bolsonaro Share 22       -0.0804      0.083     -0.972      0.331      -0.243       0.082
elderly(>70yo)            0.0402      0.005      8.395      0.000       0.031       0.050
Rural pop.               -0.0001      0.000     -0.302      0.763      -0.001       0.001
White                    -0.0015      0.009     -0.164      0.870      -0.019       0.016
Black                     0.0022      0.010      0.223      0.823      -0.017       0.022
Brown                    -0.0052      0.009     -0.560      0.575      -0.024       0.013
Indigenous               -0.0041      0.009     -0.430      0.667      -0.023       0.014
Pop. density          -1.682e-05   2.67e-05     -0.630      0.529   -6.92e-05    3.56e-05
Catholic                  1.4089      0.058     24.086      0.000       1.294       1.524
Evangelic                 1.2601      0.100     12.590      0.000       1.064       1.456
Pentecostal Evangelic     1.5374      0.151     10.156      0.000       1.240       1.834
No religion               0.9942      0.269      3.696      0.000       0.466       1.522
Education 8y less         0.0091      0.008      1.077      0.282      -0.008       0.026
Education 8-10y           0.0203      0.009      2.246      0.025       0.003       0.038
Education 11-14y          0.0127      0.009      1.437      0.151      -0.005       0.030
Education 15y plus       -0.0054      0.011     -0.504      0.615      -0.027       0.016
Income (BRL)              0.0001   2.31e-05      5.332      0.000    7.79e-05       0.000
==============================================================================
Omnibus:                       53.191   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.055
Skew:                           0.362   Prob(JB):                     4.59e-20
Kurtosis:                       4.149   Cond. No.                     3.79e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
table3 = Stargazer([model2co, model2ne, model2no, model2se, model2su ])
table3.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
print(table3.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & -0.638$^{***}$ & -0.398$^{***}$ & -0.140$^{}$ & -0.053$^{}$ & -0.080$^{}$ \\
  & (0.172) & (0.094) & (0.112) & (0.092) & (0.083) \\
 elderly(>70yo) & 0.015$^{}$ & 0.059$^{***}$ & 0.091$^{***}$ & 0.056$^{***}$ & 0.040$^{***}$ \\
  & (0.012) & (0.005) & (0.013) & (0.005) & (0.005) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.001$^{*}$ & -0.000$^{}$ & -0.000$^{}$ \\
  & (0.001) & (0.000) & (0.001) & (0.001) & (0.000) \\
 Pop. density & -0.000$^{**}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.000) \\
 White & -0.027$^{*}$ & -0.013$^{**}$ & 0.004$^{}$ & -0.015$^{*}$ & -0.002$^{}$ \\
  & (0.014) & (0.006) & (0.013) & (0.008) & (0.009) \\
 Black & -0.053$^{***}$ & -0

In [36]:
# Vaxx coverage ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y5_co, X2_co).fit()

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     2.196
Date:                Sun, 26 Mar 2023   Prob (F-statistic):            0.00403
Time:                        18:51:17   Log-Likelihood:                -2007.4
No. Observations:                 446   AIC:                             4051.
Df Residuals:                     428   BIC:                             4125.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -310.5178    260.582     -1.192      0.234    -822.697     201.662
Bolsonaro Share 22       22.8481     15.034      1.520      0.129      -6.702      52.398
elderly(>70yo)            1.2090      1.049      1.152      0.250      -0.854       3.271
Rural pop.                0.1179      0.088      1.335      0.183      -0.056       0.292
White                     0.1396      1.247      0.112      0.911      -2.311       2.590
Black                    -0.6896      1.376     -0.501      0.617      -3.395       2.015
Brown                    -0.0467      1.244     -0.038      0.970      -2.492       2.398
Indigenous               -0.4197      1.238     -0.339      0.735      -2.854       2.015
Pop. density             -0.0038      0.007     -0.535      0.593      -0.018       0.010
Catholic                 11.8801      8.123      1.463      0.144      -4.085      27.846
Evangelic               -22.1922     52.118     -0.426      0.670    -124.631      80.247
Pentecostal Evangelic    -5.4700     17.821     -0.307      0.759     -40.497      29.557
No religion              18.1826     31.651      0.574      0.566     -44.029      80.394
Education 8y less         3.7188      2.319      1.603      0.110      -0.840       8.277
Education 8-10y           3.5320      2.379      1.485      0.138      -1.143       8.207
Education 11-14y          3.6968      2.338      1.581      0.115      -0.899       8.293
Education 15y plus        3.2565      2.999      1.086      0.278      -2.638       9.151
Income (BRL)             -0.0016      0.006     -0.284      0.777      -0.013       0.010
==============================================================================
Omnibus:                       37.747   Durbin-Watson:                   2.069
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.519
Skew:                           0.527   Prob(JB):                     1.32e-15
Kurtosis:                       4.605   Cond. No.                     4.00e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
# Create the OLS model with a different name
model2ne = sm.OLS(y5_ne, X2_ne).fit()

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     14.55
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           7.93e-40
Time:                        18:51:18   Log-Likelihood:                -7939.3
No. Observations:                1787   AIC:                         1.591e+04
Df Residuals:                    1769   BIC:                         1.601e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   317.7456     55.178      5.759      0.000     209.525     425.967
Bolsonaro Share 22      -14.5672      7.863     -1.853      0.064     -29.989       0.854
elderly(>70yo)            2.1491      0.438      4.903      0.000       1.289       3.009
Rural pop.                0.0043      0.036      0.120      0.904      -0.066       0.074
White                    -1.7548      0.485     -3.616      0.000      -2.707      -0.803
Black                    -2.1315      0.519     -4.106      0.000      -3.150      -1.113
Brown                    -2.0317      0.491     -4.138      0.000      -2.995      -1.069
Indigenous               -2.2472      0.509     -4.417      0.000      -3.245      -1.249
Pop. density             -0.0024      0.001     -1.629      0.103      -0.005       0.000
Catholic                  8.6236      4.799      1.797      0.073      -0.790      18.037
Evangelic                19.2486     29.736      0.647      0.518     -39.072      77.570
Pentecostal Evangelic   -15.4642     14.573     -1.061      0.289     -44.047      13.119
No religion              -9.5345     13.337     -0.715      0.475     -35.692      16.623
Education 8y less        -0.7741      0.330     -2.344      0.019      -1.422      -0.126
Education 8-10y          -0.6684      0.446     -1.499      0.134      -1.543       0.206
Education 11-14y         -1.4434      0.439     -3.284      0.001      -2.305      -0.581
Education 15y plus       -1.2904      1.108     -1.165      0.244      -3.463       0.882
Income (BRL)              0.0112      0.004      2.739      0.006       0.003       0.019
==============================================================================
Omnibus:                      227.461   Durbin-Watson:                   1.924
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              781.199
Skew:                           0.613   Prob(JB):                    2.32e-170
Kurtosis:                       5.998   Cond. No.                     1.04e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
# Create the OLS model with a different name
model2no = sm.OLS(y5_no, X2_no).fit()

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     8.682
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           2.83e-19
Time:                        18:51:18   Log-Likelihood:                -1967.3
No. Observations:                 449   AIC:                             3971.
Df Residuals:                     431   BIC:                             4044.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -23.4738    156.310     -0.150      0.881    -330.700     283.752
Bolsonaro Share 22        2.9541      9.475      0.312      0.755     -15.669      21.577
elderly(>70yo)            4.5742      1.124      4.068      0.000       2.364       6.784
Rural pop.               -0.1632      0.070     -2.335      0.020      -0.301      -0.026
White                    -1.4902      1.107     -1.346      0.179      -3.666       0.685
Black                    -1.7764      1.192     -1.490      0.137      -4.120       0.567
Brown                    -2.0649      1.101     -1.875      0.062      -4.230       0.100
Indigenous               -2.1138      1.102     -1.918      0.056      -4.279       0.052
Pop. density             -0.0145      0.007     -2.033      0.043      -0.029      -0.000
Catholic                  8.7390      7.644      1.143      0.254      -6.284      23.762
Evangelic                -5.3640     34.560     -0.155      0.877     -73.291      62.563
Pentecostal Evangelic    -5.1556     15.723     -0.328      0.743     -36.058      25.747
No religion             -19.9880     25.765     -0.776      0.438     -70.628      30.652
Education 8y less         2.6943      1.122      2.402      0.017       0.490       4.899
Education 8-10y           2.1353      1.248      1.711      0.088      -0.318       4.589
Education 11-14y          2.8452      1.234      2.306      0.022       0.420       5.271
Education 15y plus       -1.5717      2.689     -0.585      0.559      -6.856       3.713
Income (BRL)              0.0085      0.006      1.319      0.188      -0.004       0.021
==============================================================================
Omnibus:                        0.861   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.650   Jarque-Bera (JB):                0.655
Skew:                           0.049   Prob(JB):                        0.721
Kurtosis:                       3.160   Cond. No.                     1.97e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
# Create the OLS model with a different name
model2se = sm.OLS(y5_se, X2_se).fit()

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     16.55
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           1.23e-45
Time:                        18:51:18   Log-Likelihood:                -7516.9
No. Observations:                1666   AIC:                         1.507e+04
Df Residuals:                    1648   BIC:                         1.517e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   386.6478    115.627      3.344      0.001     159.857     613.438
Bolsonaro Share 22        5.7975      8.005      0.724      0.469      -9.903      21.498
elderly(>70yo)            1.8772      0.475      3.952      0.000       0.946       2.809
Rural pop.               -0.0508      0.044     -1.145      0.252      -0.138       0.036
White                    -1.0757      0.670     -1.605      0.109      -2.390       0.239
Black                    -1.3642      0.696     -1.960      0.050      -2.729       0.001
Brown                    -1.2622      0.680     -1.857      0.064      -2.595       0.071
Indigenous               -1.0606      0.738     -1.436      0.151      -2.509       0.388
Pop. density             -0.0002      0.001     -0.360      0.719      -0.002       0.001
Catholic                  3.5669      8.138      0.438      0.661     -12.395      19.529
Evangelic                19.0762     15.844      1.204      0.229     -12.001      50.153
Pentecostal Evangelic     2.9726     14.145      0.210      0.834     -24.771      30.716
No religion            -109.2553     22.677     -4.818      0.000    -153.734     -64.776
Education 8y less        -1.9104      0.906     -2.109      0.035      -3.687      -0.134
Education 8-10y          -2.1305      0.960     -2.219      0.027      -4.014      -0.247
Education 11-14y         -2.8761      0.923     -3.115      0.002      -4.687      -1.065
Education 15y plus       -4.1500      1.075     -3.862      0.000      -6.258      -2.042
Income (BRL)              0.0047      0.003      1.832      0.067      -0.000       0.010
==============================================================================
Omnibus:                       84.036   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              290.875
Skew:                           0.078   Prob(JB):                     6.87e-64
Kurtosis:                       5.041   Cond. No.                     3.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
# Create the OLS model with a different name
model2su = sm.OLS(y5_su, X2_su).fit()

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     14.33
Date:                Sun, 26 Mar 2023   Prob (F-statistic):           9.58e-38
Time:                        18:51:18   Log-Likelihood:                -5188.1
No. Observations:                1159   AIC:                         1.041e+04
Df Residuals:                    1141   BIC:                         1.050e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   179.0237    125.565      1.426      0.154     -67.341     425.389
Bolsonaro Share 22       -8.1196      8.339     -0.974      0.330     -24.481       8.242
elderly(>70yo)            1.9489      0.483      4.038      0.000       1.002       2.896
Rural pop.                0.1276      0.047      2.691      0.007       0.035       0.221
White                     0.8419      0.925      0.910      0.363      -0.972       2.656
Black                     0.3773      1.014      0.372      0.710      -1.612       2.367
Brown                     0.7428      0.944      0.787      0.431      -1.108       2.594
Indigenous                0.4565      0.950      0.480      0.631      -1.408       2.321
Pop. density             -0.0039      0.003     -1.441      0.150      -0.009       0.001
Catholic                  1.4072      5.901      0.238      0.812     -10.171      12.985
Evangelic               -13.0265     10.097     -1.290      0.197     -32.838       6.785
Pentecostal Evangelic     4.6271     15.271      0.303      0.762     -25.335      34.589
No religion            -104.7471     27.138     -3.860      0.000    -157.994     -51.500
Education 8y less        -2.0229      0.857     -2.361      0.018      -3.704      -0.342
Education 8-10y          -2.0968      0.912     -2.298      0.022      -3.887      -0.307
Education 11-14y         -1.9184      0.893     -2.148      0.032      -3.671      -0.166
Education 15y plus       -4.1572      1.089     -3.819      0.000      -6.293      -2.021
Income (BRL)              0.0122      0.002      5.218      0.000       0.008       0.017
==============================================================================
Omnibus:                      243.439   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1841.263
Skew:                           0.754   Prob(JB):                         0.00
Kurtosis:                       8.988   Cond. No.                     3.79e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
table4 = Stargazer([model2co, model2ne, model2no, model2se, model2su ])
table4.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
print(table4.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & 22.848$^{}$ & -14.567$^{*}$ & 2.954$^{}$ & 5.798$^{}$ & -8.120$^{}$ \\
  & (15.034) & (7.863) & (9.475) & (8.005) & (8.339) \\
 elderly(>70yo) & 1.209$^{}$ & 2.149$^{***}$ & 4.574$^{***}$ & 1.877$^{***}$ & 1.949$^{***}$ \\
  & (1.049) & (0.438) & (1.124) & (0.475) & (0.483) \\
 Rural pop. & 0.118$^{}$ & 0.004$^{}$ & -0.163$^{**}$ & -0.051$^{}$ & 0.128$^{***}$ \\
  & (0.088) & (0.036) & (0.070) & (0.044) & (0.047) \\
 Pop. density & -0.004$^{}$ & -0.002$^{}$ & -0.015$^{**}$ & -0.000$^{}$ & -0.004$^{}$ \\
  & (0.007) & (0.001) & (0.007) & (0.001) & (0.003) \\
 White & 0.140$^{}$ & -1.755$^{***}$ & -1.490$^{}$ & -1.076$^{}$ & 0.842$^{}$ \\
  & (1.247) & (0.485) & (1.107) & (0.670) & (0.925) \\
 Black & -0.690$^{}$ & -2.132$^{***}

------------------------
Sensitivity tests

In [42]:
# Doses aplicadas ~ coligação PSL '18

# Create the OLS model with a different name
model1 = sm.OLS(y1, X4).fit()

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     283.5
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:18   Log-Likelihood:                -1228.2
No. Observations:                5507   AIC:                             2492.
Df Residuals:                    5489   BIC:                             2611.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.6007      0.560     -2.856      0.004      -2.699      -0.502
PSL coligation 18         0.1423      0.075      1.892      0.059      -0.005       0.290
elderly(>70yo)            0.0480      0.003     14.928      0.000       0.042       0.054
Rural pop.             3.321e-05      0.000      0.115      0.908      -0.001       0.001
White                    -0.0224      0.005     -4.888      0.000      -0.031      -0.013
Black                    -0.0236      0.005     -4.886      0.000      -0.033      -0.014
Brown                    -0.0273      0.005     -5.910      0.000      -0.036      -0.018
Indigenous               -0.0257      0.005     -5.514      0.000      -0.035      -0.017
Pop. density           -4.95e-06   7.62e-06     -0.650      0.516   -1.99e-05    9.99e-06
Catholic                  1.0384      0.037     28.405      0.000       0.967       1.110
Evangelic                 0.5908      0.092      6.431      0.000       0.411       0.771
Pentecostal Evangelic     0.9514      0.081     11.698      0.000       0.792       1.111
No religion               1.1679      0.116     10.045      0.000       0.940       1.396
Education 8y less         0.0413      0.004     11.243      0.000       0.034       0.048
Education 8-10y           0.0468      0.004     11.527      0.000       0.039       0.055
Education 11-14y          0.0511      0.004     12.494      0.000       0.043       0.059
Education 15y plus        0.0147      0.006      2.345      0.019       0.002       0.027
Income (BRL)              0.0001    1.7e-05      8.055      0.000       0.000       0.000
==============================================================================
Omnibus:                     3562.597   Durbin-Watson:                   1.453
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           553117.874
Skew:                           2.109   Prob(JB):                         0.00
Kurtosis:                      51.916   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [43]:
# Doses recebidas ~ coligação PSL '18

# Create the OLS model with a different name
model2 = sm.OLS(y2, X4).fit()

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     431.3
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:18   Log-Likelihood:                -314.46
No. Observations:                5507   AIC:                             664.9
Df Residuals:                    5489   BIC:                             784.0
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3464      0.475      4.943      0.000       1.416       3.277
PSL coligation 18         0.2828      0.064      4.439      0.000       0.158       0.408
elderly(>70yo)            0.0661      0.003     24.270      0.000       0.061       0.071
Rural pop.               -0.0012      0.000     -4.770      0.000      -0.002      -0.001
White                    -0.0201      0.004     -5.192      0.000      -0.028      -0.013
Black                    -0.0249      0.004     -6.077      0.000      -0.033      -0.017
Brown                    -0.0241      0.004     -6.152      0.000      -0.032      -0.016
Indigenous               -0.0228      0.004     -5.776      0.000      -0.031      -0.015
Pop. density          -3.298e-06   6.45e-06     -0.511      0.609    -1.6e-05    9.36e-06
Catholic                  1.2266      0.031     39.608      0.000       1.166       1.287
Evangelic                 0.6558      0.078      8.428      0.000       0.503       0.808
Pentecostal Evangelic     1.0066      0.069     14.611      0.000       0.872       1.142
No religion               1.1205      0.098     11.376      0.000       0.927       1.314
Education 8y less        -0.0024      0.003     -0.770      0.441      -0.008       0.004
Education 8-10y           0.0075      0.003      2.174      0.030       0.001       0.014
Education 11-14y          0.0054      0.003      1.547      0.122      -0.001       0.012
Education 15y plus       -0.0294      0.005     -5.543      0.000      -0.040      -0.019
Income (BRL)              0.0001   1.44e-05      7.148      0.000    7.49e-05       0.000
==============================================================================
Omnibus:                      721.811   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3396.537
Skew:                           0.554   Prob(JB):                         0.00
Kurtosis:                       6.685   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
# 3ª dose aplicada ~ coligação PSL '18

# Create the OLS model with a different name
model3 = sm.OLS(y3, X4).fit()

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     265.7
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:18   Log-Likelihood:                 7537.5
No. Observations:                5507   AIC:                        -1.504e+04
Df Residuals:                    5489   BIC:                        -1.492e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3723      0.114     -3.264      0.001      -0.596      -0.149
PSL coligation 18        -0.0705      0.015     -4.602      0.000      -0.100      -0.040
elderly(>70yo)            0.0176      0.001     26.838      0.000       0.016       0.019
Rural pop.                0.0002   5.86e-05      3.618      0.000    9.72e-05       0.000
White                    -0.0008      0.001     -0.859      0.390      -0.003       0.001
Black                    -0.0021      0.001     -2.108      0.035      -0.004      -0.000
Brown                    -0.0017      0.001     -1.819      0.069      -0.004       0.000
Indigenous               -0.0009      0.001     -0.955      0.339      -0.003       0.001
Pop. density          -1.052e-06   1.55e-06     -0.678      0.498   -4.09e-06    1.99e-06
Catholic                  0.0837      0.007     11.250      0.000       0.069       0.098
Evangelic                -0.0046      0.019     -0.244      0.808      -0.041       0.032
Pentecostal Evangelic     0.1122      0.017      6.780      0.000       0.080       0.145
No religion               0.1105      0.024      4.670      0.000       0.064       0.157
Education 8y less         0.0041      0.001      5.484      0.000       0.003       0.006
Education 8-10y           0.0054      0.001      6.565      0.000       0.004       0.007
Education 11-14y          0.0058      0.001      7.015      0.000       0.004       0.007
Education 15y plus     8.059e-06      0.001      0.006      0.995      -0.002       0.003
Income (BRL)           2.481e-05   3.47e-06      7.151      0.000     1.8e-05    3.16e-05
==============================================================================
Omnibus:                     2092.143   Durbin-Watson:                   1.788
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23795.553
Skew:                           1.488   Prob(JB):                         0.00
Kurtosis:                      12.739   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
# 3ª dose recebida ~ coligação PSL '18

# Create the OLS model with a different name
model4 = sm.OLS(y4, X4).fit()

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     328.4
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:51:18   Log-Likelihood:                 8004.2
No. Observations:                5507   AIC:                        -1.597e+04
Df Residuals:                    5489   BIC:                        -1.585e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2254      0.105     -2.151      0.032      -0.431      -0.020
PSL coligation 18        -0.0554      0.014     -3.938      0.000      -0.083      -0.028
elderly(>70yo)            0.0177      0.001     29.474      0.000       0.017       0.019
Rural pop.                0.0001   5.39e-05      2.081      0.038    6.48e-06       0.000
White                    -0.0009      0.001     -1.016      0.310      -0.003       0.001
Black                    -0.0022      0.001     -2.476      0.013      -0.004      -0.000
Brown                    -0.0018      0.001     -2.073      0.038      -0.003   -9.72e-05
Indigenous               -0.0009      0.001     -1.048      0.295      -0.003       0.001
Pop. density          -6.533e-07   1.43e-06     -0.458      0.647   -3.45e-06    2.14e-06
Catholic                  0.0974      0.007     14.245      0.000       0.084       0.111
Evangelic                 0.0061      0.017      0.354      0.724      -0.028       0.040
Pentecostal Evangelic     0.1164      0.015      7.654      0.000       0.087       0.146
No religion               0.1118      0.022      5.142      0.000       0.069       0.154
Education 8y less         0.0026      0.001      3.807      0.000       0.001       0.004
Education 8-10y           0.0040      0.001      5.311      0.000       0.003       0.006
Education 11-14y          0.0044      0.001      5.819      0.000       0.003       0.006
Education 15y plus       -0.0017      0.001     -1.456      0.145      -0.004       0.001
Income (BRL)           2.352e-05   3.19e-06      7.379      0.000    1.73e-05    2.98e-05
==============================================================================
Omnibus:                     1523.821   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10000.446
Skew:                           1.157   Prob(JB):                         0.00
Kurtosis:                       9.183   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [46]:
# Cobertura vacinal ~ coligação PSL '18

# Create the OLS model with a different name
model5 = sm.OLS(y5, X4).fit()

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     74.64
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          2.13e-232
Time:                        18:51:18   Log-Likelihood:                -24786.
No. Observations:                5507   AIC:                         4.961e+04
Df Residuals:                    5489   BIC:                         4.973e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   145.2428     40.390      3.596      0.000      66.062     224.423
PSL coligation 18        -0.9620      5.421     -0.177      0.859     -11.590       9.666
elderly(>70yo)            2.3160      0.232      9.993      0.000       1.862       2.770
Rural pop.               -0.0144      0.021     -0.694      0.488      -0.055       0.026
White                    -1.2517      0.330     -3.793      0.000      -1.899      -0.605
Black                    -1.4798      0.348     -4.248      0.000      -2.163      -0.797
Brown                    -1.4288      0.333     -4.285      0.000      -2.083      -0.775
Indigenous               -1.5665      0.336     -4.665      0.000      -2.225      -0.908
Pop. density             -0.0018      0.001     -3.362      0.001      -0.003      -0.001
Catholic                  9.3121      2.635      3.534      0.000       4.146      14.478
Evangelic                -1.3815      6.621     -0.209      0.835     -14.361      11.598
Pentecostal Evangelic    15.8784      5.862      2.709      0.007       4.387      27.370
No religion             -54.9174      8.381     -6.553      0.000     -71.347     -38.488
Education 8y less         0.3930      0.265      1.485      0.138      -0.126       0.912
Education 8-10y           0.4843      0.293      1.654      0.098      -0.090       1.059
Education 11-14y         -0.1625      0.295     -0.552      0.581      -0.740       0.415
Education 15y plus       -2.8466      0.452     -6.298      0.000      -3.733      -1.960
Income (BRL)              0.0127      0.001     10.369      0.000       0.010       0.015
==============================================================================
Omnibus:                      522.553   Durbin-Watson:                   1.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1966.115
Skew:                           0.428   Prob(JB):                         0.00
Kurtosis:                       5.799   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
# Cobertura contra Polio ~ coligação PSL '18

# Create the OLS model with a different name
model6 = sm.OLS(y6, X4).fit()

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     45.70
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          1.17e-143
Time:                        18:51:18   Log-Likelihood:                -26172.
No. Observations:                5507   AIC:                         5.238e+04
Df Residuals:                    5489   BIC:                         5.250e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   184.6538     51.951      3.554      0.000      82.809     286.498
PSL coligation 18         8.8385      6.973      1.268      0.205      -4.831      22.508
elderly(>70yo)            2.8034      0.298      9.404      0.000       2.219       3.388
Rural pop.               -0.0108      0.027     -0.405      0.685      -0.063       0.042
White                    -1.3304      0.424     -3.134      0.002      -2.163      -0.498
Black                    -1.6069      0.448     -3.587      0.000      -2.485      -0.729
Brown                    -1.5204      0.429     -3.545      0.000      -2.361      -0.680
Indigenous               -1.7742      0.432     -4.108      0.000      -2.621      -0.927
Pop. density             -0.0014      0.001     -1.948      0.051      -0.003    8.56e-06
Catholic                 10.9518      3.389      3.231      0.001       4.307      17.596
Evangelic                -9.9139      8.516     -1.164      0.244     -26.609       6.781
Pentecostal Evangelic    11.2314      7.540      1.490      0.136      -3.550      26.012
No religion             -43.5179     10.779     -4.037      0.000     -64.650     -22.386
Education 8y less         0.2107      0.340      0.619      0.536      -0.457       0.878
Education 8-10y           0.2073      0.377      0.550      0.582      -0.531       0.946
Education 11-14y         -0.4001      0.379     -1.056      0.291      -1.143       0.342
Education 15y plus       -2.9828      0.581     -5.130      0.000      -4.123      -1.843
Income (BRL)              0.0099      0.002      6.260      0.000       0.007       0.013
==============================================================================
Omnibus:                     2144.495   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34275.713
Skew:                           1.430   Prob(JB):                         0.00
Kurtosis:                      14.883   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
table5 = Stargazer([model1, model2, model3, model4, model5, model6])
table5.covariate_order(['PSL coligation 18', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
print(table5.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 PSL coligation 18 & 0.142$^{*}$ & 0.283$^{***}$ & -0.070$^{***}$ & -0.055$^{***}$ & -0.962$^{}$ & 8.839$^{}$ \\
  & (0.075) & (0.064) & (0.015) & (0.014) & (5.421) & (6.973) \\
 elderly(>70yo) & 0.048$^{***}$ & 0.066$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.316$^{***}$ & 2.803$^{***}$ \\
  & (0.003) & (0.003) & (0.001) & (0.001) & (0.232) & (0.298) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & -0.014$^{}$ & -0.011$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.027) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.022$^{***}$ & -0.020$^{***}$ & -0.001$^{}$

In [56]:
# Doses aplicadas ~ coligação PL '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X5).fit()

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     284.2
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:52:08   Log-Likelihood:                -1225.2
No. Observations:                5507   AIC:                             2486.
Df Residuals:                    5489   BIC:                             2605.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.7231      0.562     -3.068      0.002      -2.824      -0.622
PL coligation 22          0.0737      0.024      3.109      0.002       0.027       0.120
elderly(>70yo)            0.0474      0.003     14.771      0.000       0.041       0.054
Rural pop.             7.148e-05      0.000      0.248      0.804      -0.000       0.001
White                    -0.0215      0.005     -4.699      0.000      -0.031      -0.013
Black                    -0.0225      0.005     -4.643      0.000      -0.032      -0.013
Brown                    -0.0265      0.005     -5.721      0.000      -0.036      -0.017
Indigenous               -0.0249      0.005     -5.332      0.000      -0.034      -0.016
Pop. density          -4.708e-06   7.62e-06     -0.618      0.537   -1.96e-05    1.02e-05
Catholic                  1.0340      0.036     28.646      0.000       0.963       1.105
Evangelic                 0.5871      0.092      6.406      0.000       0.407       0.767
Pentecostal Evangelic     0.9713      0.081     11.974      0.000       0.812       1.130
No religion               1.1503      0.116      9.889      0.000       0.922       1.378
Education 8y less         0.0414      0.004     11.283      0.000       0.034       0.049
Education 8-10y           0.0473      0.004     11.651      0.000       0.039       0.055
Education 11-14y          0.0514      0.004     12.573      0.000       0.043       0.059
Education 15y plus        0.0152      0.006      2.438      0.015       0.003       0.027
Income (BRL)              0.0001   1.68e-05      8.501      0.000       0.000       0.000
==============================================================================
Omnibus:                     3579.915   Durbin-Watson:                   1.457
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           559884.357
Skew:                           2.124   Prob(JB):                         0.00
Kurtosis:                      52.214   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [57]:
# Doses recebidas ~ coligação PL '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X5).fit()

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     429.5
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:52:08   Log-Likelihood:                -320.84
No. Observations:                5507   AIC:                             677.7
Df Residuals:                    5489   BIC:                             796.7
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.2672      0.477      4.758      0.000       1.333       3.201
PL coligation 22          0.0531      0.020      2.637      0.008       0.014       0.093
elderly(>70yo)            0.0653      0.003     23.967      0.000       0.060       0.071
Rural pop.               -0.0012      0.000     -4.825      0.000      -0.002      -0.001
White                    -0.0194      0.004     -5.000      0.000      -0.027      -0.012
Black                    -0.0240      0.004     -5.847      0.000      -0.032      -0.016
Brown                    -0.0235      0.004     -5.983      0.000      -0.031      -0.016
Indigenous               -0.0223      0.004     -5.629      0.000      -0.030      -0.015
Pop. density          -3.256e-06   6.46e-06     -0.504      0.614   -1.59e-05    9.41e-06
Catholic                  1.2087      0.031     39.464      0.000       1.149       1.269
Evangelic                 0.6385      0.078      8.210      0.000       0.486       0.791
Pentecostal Evangelic     1.0350      0.069     15.036      0.000       0.900       1.170
No religion               1.1065      0.099     11.209      0.000       0.913       1.300
Education 8y less        -0.0023      0.003     -0.734      0.463      -0.008       0.004
Education 8-10y           0.0079      0.003      2.306      0.021       0.001       0.015
Education 11-14y          0.0057      0.003      1.636      0.102      -0.001       0.012
Education 15y plus       -0.0279      0.005     -5.269      0.000      -0.038      -0.018
Income (BRL)              0.0001   1.43e-05      7.960      0.000    8.57e-05       0.000
==============================================================================
Omnibus:                      744.369   Durbin-Watson:                   1.834
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3542.210
Skew:                           0.572   Prob(JB):                         0.00
Kurtosis:                       6.759   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
# 3ª dose aplicada ~ coligação PL '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X5).fit()

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     265.9
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:52:08   Log-Likelihood:                 7538.2
No. Observations:                5507   AIC:                        -1.504e+04
Df Residuals:                    5489   BIC:                        -1.492e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.4161      0.114     -3.638      0.000      -0.640      -0.192
PL coligation 22          0.0229      0.005      4.740      0.000       0.013       0.032
elderly(>70yo)            0.0176      0.001     26.984      0.000       0.016       0.019
Rural pop.                0.0003   5.87e-05      4.291      0.000       0.000       0.000
White                    -0.0006      0.001     -0.633      0.527      -0.002       0.001
Black                    -0.0018      0.001     -1.775      0.076      -0.004       0.000
Brown                    -0.0014      0.001     -1.533      0.125      -0.003       0.000
Indigenous               -0.0006      0.001     -0.636      0.525      -0.002       0.001
Pop. density          -8.926e-07   1.55e-06     -0.575      0.565   -3.93e-06    2.15e-06
Catholic                  0.0917      0.007     12.471      0.000       0.077       0.106
Evangelic                 0.0036      0.019      0.193      0.847      -0.033       0.040
Pentecostal Evangelic     0.1094      0.017      6.625      0.000       0.077       0.142
No religion               0.1059      0.024      4.470      0.000       0.059       0.152
Education 8y less         0.0041      0.001      5.523      0.000       0.003       0.006
Education 8-10y           0.0055      0.001      6.670      0.000       0.004       0.007
Education 11-14y          0.0059      0.001      7.051      0.000       0.004       0.007
Education 15y plus       -0.0005      0.001     -0.421      0.674      -0.003       0.002
Income (BRL)           2.261e-05   3.43e-06      6.594      0.000    1.59e-05    2.93e-05
==============================================================================
Omnibus:                     2102.722   Durbin-Watson:                   1.797
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24030.889
Skew:                           1.496   Prob(JB):                         0.00
Kurtosis:                      12.786   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
# 3ª dose recebida ~ coligação PL '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X5).fit()

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.505
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     329.3
Date:                Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:52:08   Log-Likelihood:                 8008.1
No. Observations:                5507   AIC:                        -1.598e+04
Df Residuals:                    5489   BIC:                        -1.586e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2657      0.105     -2.530      0.011      -0.472      -0.060
PL coligation 22          0.0214      0.004      4.824      0.000       0.013       0.030
elderly(>70yo)            0.0178      0.001     29.603      0.000       0.017       0.019
Rural pop.                0.0001   5.39e-05      2.721      0.007     4.1e-05       0.000
White                    -0.0007      0.001     -0.781      0.435      -0.002       0.001
Black                    -0.0019      0.001     -2.134      0.033      -0.004      -0.000
Brown                    -0.0015      0.001     -1.783      0.075      -0.003       0.000
Indigenous               -0.0006      0.001     -0.728      0.466      -0.002       0.001
Pop. density          -5.123e-07   1.42e-06     -0.360      0.719    -3.3e-06    2.28e-06
Catholic                  0.1040      0.007     15.405      0.000       0.091       0.117
Evangelic                 0.0128      0.017      0.750      0.453      -0.021       0.046
Pentecostal Evangelic     0.1146      0.015      7.557      0.000       0.085       0.144
No religion               0.1074      0.022      4.938      0.000       0.065       0.150
Education 8y less         0.0026      0.001      3.850      0.000       0.001       0.004
Education 8-10y           0.0041      0.001      5.429      0.000       0.003       0.006
Education 11-14y          0.0045      0.001      5.867      0.000       0.003       0.006
Education 15y plus       -0.0022      0.001     -1.841      0.066      -0.004       0.000
Income (BRL)           2.183e-05   3.15e-06      6.933      0.000    1.57e-05     2.8e-05
==============================================================================
Omnibus:                     1520.185   Durbin-Watson:                   1.799
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9920.709
Skew:                           1.156   Prob(JB):                         0.00
Kurtosis:                       9.156   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [60]:
# Cobertura vacinal ~ coligação PL '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X5).fit()

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     74.72
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          1.21e-232
Time:                        18:52:08   Log-Likelihood:                -24785.
No. Observations:                5507   AIC:                         4.961e+04
Df Residuals:                    5489   BIC:                         4.973e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   141.9337     40.497      3.505      0.000      62.544     221.323
PL coligation 22          1.8571      1.710      1.086      0.278      -1.496       5.210
elderly(>70yo)            2.3116      0.232      9.985      0.000       1.858       2.765
Rural pop.               -0.0123      0.021     -0.594      0.553      -0.053       0.028
White                    -1.2324      0.330     -3.729      0.000      -1.880      -0.585
Black                    -1.4525      0.349     -4.159      0.000      -2.137      -0.768
Brown                    -1.4074      0.334     -4.214      0.000      -2.062      -0.753
Indigenous               -1.5440      0.336     -4.591      0.000      -2.203      -0.885
Pop. density             -0.0018      0.001     -3.345      0.001      -0.003      -0.001
Catholic                  9.5699      2.603      3.677      0.000       4.467      14.672
Evangelic                -1.1062      6.608     -0.167      0.867     -14.061      11.848
Pentecostal Evangelic    16.0230      5.850      2.739      0.006       4.556      27.490
No religion             -55.3282      8.389     -6.596      0.000     -71.773     -38.883
Education 8y less         0.3958      0.265      1.496      0.135      -0.123       0.915
Education 8-10y           0.4943      0.293      1.687      0.092      -0.080       1.069
Education 11-14y         -0.1574      0.295     -0.534      0.593      -0.735       0.420
Education 15y plus       -2.8614      0.451     -6.347      0.000      -3.745      -1.978
Income (BRL)              0.0127      0.001     10.484      0.000       0.010       0.015
==============================================================================
Omnibus:                      523.102   Durbin-Watson:                   1.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1967.260
Skew:                           0.429   Prob(JB):                         0.00
Kurtosis:                       5.799   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [61]:
# Cobertura contra Polio ~ coligação PL '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X5).fit()

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     46.04
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          9.89e-145
Time:                        18:52:08   Log-Likelihood:                -26169.
No. Observations:                5507   AIC:                         5.237e+04
Df Residuals:                    5489   BIC:                         5.249e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   175.1409     52.069      3.364      0.001      73.064     277.217
PL coligation 22          5.6673      2.199      2.577      0.010       1.356       9.979
elderly(>70yo)            2.7637      0.298      9.284      0.000       2.180       3.347
Rural pop.               -0.0074      0.027     -0.276      0.783      -0.060       0.045
White                    -1.2662      0.425     -2.980      0.003      -2.099      -0.433
Black                    -1.5209      0.449     -3.387      0.001      -2.401      -0.641
Brown                    -1.4556      0.429     -3.390      0.001      -2.297      -0.614
Indigenous               -1.7104      0.432     -3.955      0.000      -2.558      -0.863
Pop. density             -0.0014      0.001     -1.920      0.055      -0.003    2.82e-05
Catholic                 10.7797      3.347      3.221      0.001       4.219      17.340
Evangelic               -10.0305      8.497     -1.181      0.238     -26.687       6.626
Pentecostal Evangelic    12.5945      7.521      1.675      0.094      -2.150      27.339
No religion             -44.8566     10.786     -4.159      0.000     -66.001     -23.712
Education 8y less         0.2203      0.340      0.648      0.517      -0.447       0.887
Education 8-10y           0.2445      0.377      0.649      0.516      -0.494       0.983
Education 11-14y         -0.3782      0.379     -0.999      0.318      -1.121       0.364
Education 15y plus       -2.9550      0.580     -5.098      0.000      -4.091      -1.819
Income (BRL)              0.0103      0.002      6.577      0.000       0.007       0.013
==============================================================================
Omnibus:                     2136.255   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34117.010
Skew:                           1.423   Prob(JB):                         0.00
Kurtosis:                      14.857   Cond. No.                     2.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [62]:
table6 = Stargazer([model1, model2, model3, model4, model5, model6])
table6.covariate_order(['PL coligation 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
print(table6.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 PL coligation 22 & 0.074$^{***}$ & 0.053$^{***}$ & 0.023$^{***}$ & 0.021$^{***}$ & 1.857$^{}$ & 5.667$^{***}$ \\
  & (0.024) & (0.020) & (0.005) & (0.004) & (1.710) & (2.199) \\
 elderly(>70yo) & 0.047$^{***}$ & 0.065$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.312$^{***}$ & 2.764$^{***}$ \\
  & (0.003) & (0.003) & (0.001) & (0.001) & (0.232) & (0.298) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{***}$ & -0.012$^{}$ & -0.007$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.027) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.022$^{***}$ & -0.019$^{***}$ & -0.001$^{

------------------------
Descriptive statistics

In [63]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Far-left 22', 'Far-right 22',
       'PL coligation 22', 'Vaccination coverage', 'BCG coverage',
       'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black',
       'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)', 'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [64]:
print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  5507.0 &     0.439 &    0.187 &    0.061 &      0.890 \\
Far-left 18           &  5507.0 &     0.004 &    0.002 &    0.000 &      0.025 \\
Far-right 22          &  5507.0 &     0.004 &    0.004 &    0.000 &      0.032 \\
Dose1 Applied         &  5507.0 &     0.754 &    0.176 &    0.009 &      3.999 \\
Dose2 Applied         &  5507.0 &     0.620 &    0.182 &    0.000 &      3.256 \\
Dose3 Applied         &  5507.0 &     0.139 &    0.083 &    0.000 &      0.827 \\
Dose1 Received        &  5507.0 &     0.793 &    0.159 &    0.179 &      3.120 \\
Dose2 Received        &  5507.0 &     0.648 &    0.172 &    0.085 &      2.811 \\
Dose3 Received        &  5507.0 &     0.140 &    0.080 &    0.004 &      0.744 \\
Vaccination coverage  &  5507.0 &    76.939 &   24.189 &    2.280 &    245.580 \\
BCG coverage          &  5507.0 &    73.169 &   36.321 &    0.000 &    772.

<ipython-input-64-c504c304be65>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [65]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [66]:
filtered_df = df[df['Region'] == 'Norte']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  449.0 &     0.454 &    0.166 &    0.086 &     0.829 \\
Far-left 18           &  449.0 &     0.005 &    0.003 &    0.000 &     0.020 \\
Far-right 22          &  449.0 &     0.001 &    0.001 &    0.000 &     0.006 \\
Dose1 Applied         &  449.0 &     0.617 &    0.164 &    0.175 &     2.252 \\
Dose2 Applied         &  449.0 &     0.446 &    0.145 &    0.051 &     1.197 \\
Dose3 Applied         &  449.0 &     0.068 &    0.047 &    0.001 &     0.406 \\
Dose1 Received        &  449.0 &     0.617 &    0.173 &    0.179 &     1.636 \\
Dose2 Received        &  449.0 &     0.445 &    0.151 &    0.085 &     0.961 \\
Dose3 Received        &  449.0 &     0.066 &    0.043 &    0.004 &     0.309 \\
Vaccination coverage  &  449.0 &    68.026 &   22.440 &   19.030 &   136.230 \\
BCG coverage          &  449.0 &    75.867 &   31.241 &    5.130 &   235.470 \\
Polio coverage   

<ipython-input-66-ff39d407ecb8>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [67]:
filtered_df = df[df['Region'] == 'Nordeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &     mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1787.0 &    0.235 &    0.089 &    0.061 &     0.623 \\
Far-left 18           &  1787.0 &    0.003 &    0.002 &    0.000 &     0.022 \\
Far-right 22          &  1787.0 &    0.002 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  1787.0 &    0.676 &    0.227 &    0.009 &     3.999 \\
Dose2 Applied         &  1787.0 &    0.530 &    0.205 &    0.000 &     3.256 \\
Dose3 Applied         &  1787.0 &    0.101 &    0.066 &    0.000 &     0.827 \\
Dose1 Received        &  1787.0 &    0.750 &    0.148 &    0.269 &     3.120 \\
Dose2 Received        &  1787.0 &    0.581 &    0.157 &    0.113 &     2.811 \\
Dose3 Received        &  1787.0 &    0.102 &    0.060 &    0.007 &     0.690 \\
Vaccination coverage  &  1787.0 &   68.263 &   21.967 &    5.100 &   214.260 \\
BCG coverage          &  1787.0 &   64.670 &   39.712 &    0.000 &   772.860 \\
Polio coverage   

<ipython-input-67-1965c469d4bc>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [68]:
filtered_df = df[df['Region'] == 'Sudeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  1666.0 &     0.511 &    0.129 &    0.155 &      0.812 \\
Far-left 18           &  1666.0 &     0.004 &    0.003 &    0.000 &      0.025 \\
Far-right 22          &  1666.0 &     0.007 &    0.004 &    0.000 &      0.023 \\
Dose1 Applied         &  1666.0 &     0.806 &    0.093 &    0.420 &      1.609 \\
Dose2 Applied         &  1666.0 &     0.681 &    0.123 &    0.172 &      1.395 \\
Dose3 Applied         &  1666.0 &     0.165 &    0.077 &    0.001 &      0.741 \\
Dose1 Received        &  1666.0 &     0.832 &    0.125 &    0.437 &      1.658 \\
Dose2 Received        &  1666.0 &     0.700 &    0.140 &    0.229 &      1.498 \\
Dose3 Received        &  1666.0 &     0.166 &    0.075 &    0.010 &      0.744 \\
Vaccination coverage  &  1666.0 &    80.422 &   23.858 &    2.280 &    210.880 \\
BCG coverage          &  1666.0 &    73.449 &   32.653 &    0.000 &    230.

<ipython-input-68-77f52f9ad7b6>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [69]:
filtered_df = df[df['Region'] == 'Sul']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1159.0 &     0.598 &    0.115 &    0.282 &     0.890 \\
Far-left 18           &  1159.0 &     0.004 &    0.002 &    0.000 &     0.024 \\
Far-right 22          &  1159.0 &     0.005 &    0.003 &    0.000 &     0.032 \\
Dose1 Applied         &  1159.0 &     0.842 &    0.099 &    0.396 &     2.252 \\
Dose2 Applied         &  1159.0 &     0.734 &    0.108 &    0.272 &     1.950 \\
Dose3 Applied         &  1159.0 &     0.188 &    0.083 &    0.006 &     0.503 \\
Dose1 Received        &  1159.0 &     0.864 &    0.129 &    0.377 &     2.575 \\
Dose2 Received        &  1159.0 &     0.751 &    0.131 &    0.296 &     2.177 \\
Dose3 Received        &  1159.0 &     0.189 &    0.082 &    0.022 &     0.509 \\
Vaccination coverage  &  1159.0 &    87.193 &   23.442 &    5.760 &   245.580 \\
BCG coverage          &  1159.0 &    83.753 &   36.360 &    0.000 &   300.000 \\
Polio

<ipython-input-69-f4e8c2d7bdf7>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [70]:
filtered_df = df[df['Region'] == 'Centro-Oeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

<ipython-input-70-a0e93a3389ef>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  446.0 &     0.564 &    0.101 &    0.174 &     0.811 \\
Far-left 18           &  446.0 &     0.004 &    0.002 &    0.000 &     0.014 \\
Far-right 22          &  446.0 &     0.003 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  446.0 &     0.785 &    0.134 &    0.370 &     2.081 \\
Dose2 Applied         &  446.0 &     0.635 &    0.142 &    0.304 &     1.730 \\
Dose3 Applied         &  446.0 &     0.135 &    0.076 &    0.000 &     0.447 \\
Dose1 Received        &  446.0 &     0.809 &    0.184 &    0.378 &     2.484 \\
Dose2 Received        &  446.0 &     0.655 &    0.171 &    0.297 &     1.990 \\
Dose3 Received        &  446.0 &     0.136 &    0.072 &    0.015 &     0.461 \\
Vaccination coverage  &  446.0 &    81.013 &   22.759 &   21.360 &   196.690 \\
BCG coverage          &  446.0 &    75.959 &   31.371 &    4.000 &   220.000 \\
Polio coverage   